In [1]:
import pandas as pd
import numpy as np
import joblib as jb
from scipy.stats import spearmanr
%matplotlib inline

In [2]:
train_train_level0 = jb.load("train_train_level0.pkl.z")
train_train_level1 = jb.load("train_train_level1.pkl.z")
train_valid_level1 = jb.load("train_valid_level1.pkl.z")
test = pd.read_feather("test.f")


X_train0, y_train0 = train_train_level0.filter(regex=r'feature', axis=1), train_train_level0['target_kazutsugi']
X_train1, y_train1 = train_train_level1.filter(regex=r'feature', axis=1), train_train_level1['target_kazutsugi']
X_val1, y_val1 = train_valid_level1.filter(regex=r'feature', axis=1), train_valid_level1['target_kazutsugi']
X_test = test.filter(regex=r'feature', axis=1)

# Manual Depth

In [3]:
%%time
from xgboost import XGBRegressor


for depth in [1, 5, 10]:
    mdl = XGBRegressor(max_depth=depth, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, tree_method='gpu_hist', gpu_id=0)
    mdl.fit(X_train0, y_train0)

    p = mdl.predict(X_train1)
    model_name_train1 = "./preds_train1/xgb_depth_{}.pkl.z".format(depth)
    jb.dump(p, model_name_train1)

    metric = spearmanr(y_train1.values,p).correlation

    p = mdl.predict(X_val1)
    model_name_val1 = "./preds_val1/xgb_depth_{}.pkl.z".format(depth)
    jb.dump(p, model_name_val1)

    p = mdl.predict(X_test)
    model_name_test = "./preds_test/xgb_depth_{}.pkl.z".format(depth)
    jb.dump(p, model_name_test)

    print(metric)

0.025422023331943475
0.038765243262175154
0.03257493292028262
CPU times: user 7min 49s, sys: 12.3 s, total: 8min 1s
Wall time: 2min 33s


# Random Hypers

In [ ]:
%%time
from xgboost import XGBRegressor

def gen_xgb(seed):
    np.random.seed(seed)
    depth = np.random.randint(1,11)
    learning_rate = np.random.uniform(1e-3, 5e-2)
    n_estimators = np.random.randint(100, 5000)
    colsample_bytree = np.random.uniform(0.05, 0.95)
    subsample = np.random.uniform(0.05, 0.95)
    
    mdl = XGBRegressor(max_depth=depth, learning_rate=learning_rate, n_estimators=n_estimators, colsample_bytree=colsample_bytree, 
                       subsample=subsample, tree_method='gpu_hist', gpu_id=0, seed=seed)
    mdl.fit(X_train0, y_train0)

    p = mdl.predict(X_train1)
    model_name_train1 = "./preds_train1/xgb_rhyper_{}.pkl.z".format(seed)
    jb.dump(p, model_name_train1)

    metric = spearmanr(y_train1.values,p).correlation

    p = mdl.predict(X_val1)
    model_name_val1 = "./preds_val1/xgb_rhyper_{}.pkl.z".format(seed)
    jb.dump(p, model_name_val1)

    p = mdl.predict(X_test)
    model_name_test = "./preds_test/xgb_rhyper_{}.pkl.z".format(seed)
    jb.dump(p, model_name_test)

    print(seed, depth, learning_rate, n_estimators, colsample_bytree, subsample, metric)
    
for seed in range(50):
    gen_xgb(seed)